In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [37]:
raw = pd.read_csv("../data/de_party_voted_78_raw.csv")
pd.set_option('display.max_columns', 500)

In [38]:
raw.head()

,cntry,cname,cedition,cproddat,cseqno,name,essround,edition,idno,dweight,pspwght,pweight,prtvede1
0,DE,ESS1-8e01,1.0,12.12.2018,84321,ESS7e02_2,7,2.2,10000011,0.531346,0.607555,2.304126,2
1,DE,ESS1-8e01,1.0,12.12.2018,84322,ESS7e02_2,7,2.2,10000021,0.531346,0.462666,2.304126,1
2,DE,ESS1-8e01,1.0,12.12.2018,84323,ESS7e02_2,7,2.2,10000051,1.227808,0.640290,2.304126,1
3,DE,ESS1-8e01,1.0,12.12.2018,84324,ESS7e02_2,7,2.2,10000061,1.227808,1.142616,2.304126,1
4,DE,ESS1-8e01,1.0,12.12.2018,84325,ESS7e02_2,7,2.2,10000111,1.227808,0.852731,2.304126,1


In [39]:
raw.columns

Index(['cntry', 'cname', 'cedition', 'cproddat', 'cseqno', 'name', 'essround',
       'edition', 'idno', 'dweight', 'pspwght', 'pweight', 'prtvede1'],
      dtype='object')

In [40]:
de_parties = raw.drop(columns=['cname', 
                  'cedition', 
                  'cproddat', 
                  'cseqno', 
                  'name',
                  'edition',
                  'dweight', 
                  'pspwght', 
                  'pweight'])

In [41]:
de_parties.head()

,cntry,essround,idno,prtvede1
0,DE,7,10000011,2
1,DE,7,10000021,1
2,DE,7,10000051,1
3,DE,7,10000061,1
4,DE,7,10000111,1


In [42]:
de_parties = de_parties.rename(columns={"cntry":"country",
                                        "essround":"round",
                                        "prtvede1":"party"})

In [43]:
de_parties.head()

,country,round,idno,party
0,DE,7,10000011,2
1,DE,7,10000021,1
2,DE,7,10000051,1
3,DE,7,10000061,1
4,DE,7,10000111,1


In [44]:
de_parties.shape

(5897, 4)

In [45]:
de_parties["party"].value_counts()

1     1595
66    1439
2     1239
4      394
3      357
77     282
88     259
6      120
5       97
9       60
7       37
8       18
Name: party, dtype: int64

### Possible line of analysis: understanding extreme-right voters. For that, we´ll get the idno of the individuals who voted for AfD (#6) and NPD(#8) in Germany: 120 + 18

In [46]:
afd_npd = de_parties[de_parties["party"].isin([6,8])]

In [47]:
#We have our list with the ids of those voters
afd_npd_ids = afd_npd["idno"].to_list()

In [48]:
#Now we´re gonna check at their human values scale

de_hv = pd.read_csv("../data/de_hv_leftright_parties_7_8_df.csv")

In [49]:
de_hv.shape

(5800, 28)

In [50]:
de_hv_ext = de_hv[de_hv["idno"].isin(afd_npd_ids)]

In [51]:
de_hv_ext.shape

(138, 28)

### We divide our df in 3: 
### de_ext: those who voted for AfD (p_voted==6) or NPD (p_voted==8) 
### de_novot: those who didn´t vote (p_voted == 66)
### de_noext: the rest (also excluding those who didn´t answer(77) or didn´t know(88)

In [52]:
#We import the df with the human values
de_hv = pd.read_csv("../data/de_hv_leftright_parties_7_8_df.csv")

In [53]:
de_hv.head()

,const,country_x,round_x,idno,new_ideas,rich,equality,abilities,safety,new_diff,obey,understand_diff,modesty,good_time,freedom,help_others,success,strong_gov,adventures,behave,get_respect,loyalty,nature,traditions,fun_pleasure,left_right,p_voted,p_close
0,1.0,DE,7,10000011,2.0,4.0,1.0,3.0,2.0,3.0,3.0,2.0,3.0,2.0,1.0,2.0,4.0,2.0,2.0,2.0,3.0,1.0,2.0,2.0,4.0,4,2,66
1,1.0,DE,7,10000021,2.0,5.0,2.0,5.0,3.0,2.0,2.0,2.0,2.0,4.0,2.0,1.0,3.0,2.0,5.0,2.0,5.0,2.0,2.0,1.0,2.0,5,1,1
2,1.0,DE,7,10000051,4.0,5.0,1.0,3.0,2.0,3.0,5.0,2.0,2.0,3.0,2.0,3.0,3.0,2.0,4.0,2.0,4.0,2.0,2.0,4.0,3.0,6,1,66
3,1.0,DE,7,10000061,2.0,6.0,2.0,5.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,5.0,6.0,3.0,5.0,1.0,1.0,2.0,3.0,5,1,66
4,1.0,DE,7,10000111,1.0,5.0,2.0,3.0,3.0,4.0,4.0,2.0,3.0,4.0,2.0,2.0,5.0,4.0,4.0,6.0,4.0,2.0,3.0,2.0,4.0,5,1,1


In [54]:
de_ext = de_hv[de_hv["p_voted"].isin([6,8])]

In [55]:
de_novot = de_hv[de_hv["p_voted"] == 66]

In [56]:
de_noext = de_hv[~de_hv["p_voted"].isin([6,8,66,77,88])]

In [57]:
de_ext.describe()

,const,round_x,idno,new_ideas,rich,equality,abilities,safety,new_diff,obey,understand_diff,modesty,good_time,freedom,help_others,success,strong_gov,adventures,behave,get_respect,loyalty,nature,traditions,fun_pleasure,left_right,p_voted,p_close
count,136.0,136.000000,1.360000e+02,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000
mean,1.0,7.411765,1.059291e+07,2.323529,4.360294,2.213235,3.544118,2.205882,2.823529,3.441176,2.308824,2.661765,2.205882,1.764706,1.970588,2.963235,1.816176,4.073529,2.816176,3.477941,1.544118,2.051471,2.830882,2.970588,7.014706,6.264706,35.286765
std,0.0,0.493972,1.823896e+06,1.354730,1.126600,1.137952,1.475065,1.199492,1.310258,1.543276,1.014814,1.266202,1.129521,0.871230,1.039524,1.379254,1.090037,1.314491,1.345512,1.355193,0.630903,0.937450,1.432714,1.429536,10.101694,0.680254,30.409284
min,1.0,7.000000,1.000005e+07,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,1.000000
25%,1.0,7.000000,1.000833e+07,1.000000,4.000000,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000,5.000000,6.000000,6.000000
50%,1.0,7.000000,1.003091e+07,2.000000,4.000000,2.000000,3.500000,2.000000,3.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,1.000000,4.000000,3.000000,3.000000,1.000000,2.000000,3.000000,3.000000,5.000000,6.000000,9.000000
75%,1.0,8.000000,1.006970e+07,3.000000,5.000000,3.000000,5.000000,3.000000,4.000000,5.000000,3.000000,3.000000,3.000000,2.000000,2.000000,4.000000,2.000000,5.000000,4.000000,5.000000,2.000000,3.000000,4.000000,4.000000,7.000000,6.000000,66.000000
max,1.0,8.000000,1.930004e+07,8.000000,6.000000,6.000000,8.000000,6.000000,6.000000,6.000000,6.000000,8.000000,6.000000,6.000000,8.000000,6.000000,5.000000,6.000000,6.000000,7.000000,3.000000,5.000000,6.000000,6.000000,88.000000,8.000000,66.000000


In [58]:
de_noext.describe()

,const,round_x,idno,new_ideas,rich,equality,abilities,safety,new_diff,obey,understand_diff,modesty,good_time,freedom,help_others,success,strong_gov,adventures,behave,get_respect,loyalty,nature,traditions,fun_pleasure,left_right,p_voted,p_close
count,3712.0,3712.000000,3.712000e+03,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.00000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000
mean,1.0,7.463901,1.049964e+07,2.380388,4.529095,1.985183,3.717942,2.508890,3.035022,3.587554,2.021013,2.580011,2.479795,1.906789,1.960129,3.155172,2.330550,4.384968,3.00458,3.500269,1.598599,1.991918,2.899246,3.237608,5.483028,2.119073,24.603987
std,0.0,0.498762,1.580119e+06,1.183158,1.132892,1.044701,1.362951,1.286678,1.359364,1.452708,0.899469,1.187176,1.156389,0.964717,0.872357,1.307341,1.220658,1.294734,1.33914,1.349471,0.737039,0.959115,1.393320,1.331422,9.466117,1.472589,30.412417
min,1.0,7.000000,1.000000e+07,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,1.0,7.000000,1.000549e+07,2.000000,4.000000,1.000000,3.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000,1.000000,3.000000,2.00000,2.000000,1.000000,1.000000,2.000000,2.000000,3.000000,1.000000,2.000000
50%,1.0,7.000000,1.002135e+07,2.000000,5.000000,2.000000,4.000000,2.000000,3.000000,4.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,2.000000,5.000000,3.00000,3.000000,2.000000,2.000000,3.000000,3.000000,5.000000,2.000000,3.000000
75%,1.0,8.000000,1.006652e+07,3.000000,5.000000,2.000000,5.000000,3.000000,4.000000,5.000000,2.000000,3.000000,3.000000,2.000000,2.000000,4.000000,3.000000,5.000000,4.00000,5.000000,2.000000,2.000000,4.000000,4.000000,5.000000,3.000000,66.000000
max,1.0,8.000000,1.930005e+07,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.00000,8.000000,8.000000,8.000000,8.000000,8.000000,88.000000,9.000000,88.000000


In [59]:
de_novot.describe()
#Note: the leftright_scale mean is 13 because many people replied 88 (don´t know)
#The scale for left_right goes: 0--> left, 10--> right

,const,round_x,idno,new_ideas,rich,equality,abilities,safety,new_diff,obey,understand_diff,modesty,good_time,freedom,help_others,success,strong_gov,adventures,behave,get_respect,loyalty,nature,traditions,fun_pleasure,left_right,p_voted,p_close
count,1415.0,1415.000000,1.415000e+03,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.000000,1415.0,1415.000000
mean,1.0,7.509541,1.051711e+07,2.496820,4.286926,2.065724,3.401413,2.553357,2.862191,3.718021,2.235336,2.823322,2.395053,2.074205,2.107420,3.124382,2.438869,3.926502,3.032509,3.530035,1.732155,2.358304,3.203534,2.967491,10.781625,66.0,44.030389
std,0.0,0.500086,1.634516e+06,1.368399,1.362643,1.231622,1.484955,1.421626,1.446873,1.552133,1.171513,1.424634,1.278747,1.251084,1.133167,1.453696,1.467339,1.572473,1.421820,1.459610,1.074814,1.282385,1.599899,1.434201,22.007385,0.0,30.182562
min,1.0,7.000000,1.000002e+07,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,66.0,1.000000
25%,1.0,7.000000,1.000535e+07,2.000000,3.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,1.000000,3.000000,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000,3.000000,66.0,4.000000
50%,1.0,8.000000,1.001403e+07,2.000000,5.000000,2.000000,3.000000,2.000000,3.000000,4.000000,2.000000,3.000000,2.000000,2.000000,2.000000,3.000000,2.000000,4.000000,3.000000,3.000000,2.000000,2.000000,3.000000,3.000000,5.000000,66.0,66.000000
75%,1.0,8.000000,1.006514e+07,3.000000,5.000000,2.000000,5.000000,3.000000,4.000000,5.000000,3.000000,4.000000,3.000000,3.000000,3.000000,4.000000,3.000000,5.000000,4.000000,5.000000,2.000000,3.000000,4.000000,4.000000,5.000000,66.0,66.000000
max,1.0,8.000000,1.930005e+07,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,88.000000,66.0,88.000000
